In [ ]:
import cv2
import mediapipe as mp
import os
import numpy as np

In [ ]:
# Initialize MediaPipe Holistic and Hands models
mp_holistic = mp.solutions.holistic
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)

video_path = "C:\\Users\DELL\Desktop\Pose_extraction\\videos"
output_dir = "images"


In [ ]:
# Load video file
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for file_name in sorted(os.listdir(video_path)):

    file_path = os.path.join(video_path, file_name)
    # Open the video file
    cap = cv2.VideoCapture(file_path)


    # Define variables for output file names and frame counter
    output_prefix = "pose_frame_"
    output_suffix = ".png"
    frame_num = 0
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    interval = 0.5 # Save a pose every second
    next_frame_save = fps * interval

    # Loop over video frames
    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()
        if not success:
            break

        # Convert frame to RGB format
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Use MediaPipe Holistic to detect poses in the frame
        results_holistic = holistic.process(frame_rgb)

        # Use MediaPipe Hands to detect hands in the frame
        results_hands = hands.process(frame_rgb)

        # Extract pose landmarks from Holistic results
        pose_landmarks = results_holistic.pose_landmarks

        # Extract hand landmarks from Hands results
        hand_landmarks = results_hands.multi_hand_landmarks

        # Save pose and hand landmarks as an image file
        if frame_num == next_frame_save and pose_landmarks is not None:
            # Create white image with same size as video frame
            white_img = 255 * np.ones_like(frame)
            # Draw pose landmarks on white image
            mp_drawing.draw_landmarks(
                white_img, pose_landmarks, mp_holistic.POSE_CONNECTIONS)
            # Draw hand landmarks on white image
            if hand_landmarks:
                for hand_lmks in hand_landmarks:
                    mp_drawing.draw_landmarks(
                        white_img, hand_lmks, mp_hands.HAND_CONNECTIONS)
            # Create output file path and name
            output_path = os.path.join(
                output_dir, f"{file_name}{frame_num:04d}{output_suffix}")
            # Save the white image as an output file
            cv2.imwrite(output_path, white_img)

            next_frame_save += fps * interval

        # Increment frame counter
        frame_num += 1

# Clean up
cap.release()
holistic.close()
hands.close()